In [0]:
# read only instance
dbutils.widgets.text("url", "neo4j+s://demo.neo4jlabs.com")
dbutils.widgets.text("user", "fincen")
dbutils.widgets.text("password", "fincen")
dbutils.widgets.text("database", "fincen")

In [0]:
import urllib.request

file = "download_transactions_map.csv"
url = "https://raw.githubusercontent.com/neo4j-graph-examples/fincen/main/import/" + file

dbutils.fs.put(file, str(urllib.request.urlopen(url).read(), encoding="utf-8"), True)
csv = spark.read.csv(file, header=True, inferSchema=True)
display(csv.take(10))

id,icij_sar_id,filer_org_name_id,filer_org_name,begin_date,end_date,originator_bank_id,originator_bank,originator_bank_country,originator_iso,beneficiary_bank_id,beneficiary_bank,beneficiary_bank_country,beneficiary_iso,number_transactions,amount_transactions
223254,3297,the-bank-of-new-york-mellon-corp,The Bank of New York Mellon Corp.,"Mar 25, 2015","Sep 25, 2015",cimb-bank-berhad,CIMB Bank Berhad,Singapore,SGP,barclays-bank-plc-london-england-gbr,Barclays Bank Plc,United Kingdom,GBR,68,5.689852347E7
223255,3297,the-bank-of-new-york-mellon-corp,The Bank of New York Mellon Corp.,"Mar 30, 2015","Sep 25, 2015",cimb-bank-berhad,CIMB Bank Berhad,Singapore,SGP,barclays-bank-plc-london-england-gbr,Barclays Bank Plc,United Kingdom,GBR,118,1.1623836125E8
223258,2924,the-bank-of-new-york-mellon-corp,The Bank of New York Mellon Corp.,"Jul 5, 2012","Jul 5, 2012",barclays-bank-plc-ho-uk,Barclays Bank Plc Ho UK,United Kingdom,GBR,skandinaviska-enskilda-banken-stockholm-sweden-swe,Skandinaviska Enskilda Banken,Sweden,SWE,null,5000.0
223259,2924,the-bank-of-new-york-mellon-corp,The Bank of New York Mellon Corp.,"Jun 20, 2012","Jun 20, 2012",barclays-bank-plc-ho-uk,Barclays Bank Plc Ho UK,United Kingdom,GBR,skandinaviska-enskilda-banken-stockholm-sweden-swe,Skandinaviska Enskilda Banken,Sweden,SWE,null,9990.0
223260,2924,the-bank-of-new-york-mellon-corp,The Bank of New York Mellon Corp.,"May 31, 2012","May 31, 2012",barclays-bank-plc-ho-uk,Barclays Bank Plc Ho UK,United Kingdom,GBR,skandinaviska-enskilda-banken-stockholm-sweden-swe,Skandinaviska Enskilda Banken,Sweden,SWE,null,12000.0
223261,2924,the-bank-of-new-york-mellon-corp,The Bank of New York Mellon Corp.,"May 29, 2012","May 29, 2012",barclays-bank-plc-ho-uk,Barclays Bank Plc Ho UK,United Kingdom,GBR,skandinaviska-enskilda-banken-stockholm-sweden-swe,Skandinaviska Enskilda Banken,Sweden,SWE,null,88.03
223262,2924,the-bank-of-new-york-mellon-corp,The Bank of New York Mellon Corp.,"May 29, 2012","May 29, 2012",barclays-bank-plc-ho-uk,Barclays Bank Plc Ho UK,United Kingdom,GBR,skandinaviska-enskilda-banken-stockholm-sweden-swe,Skandinaviska Enskilda Banken,Sweden,SWE,null,13000.0
223263,2924,the-bank-of-new-york-mellon-corp,The Bank of New York Mellon Corp.,"May 22, 2012","May 22, 2012",barclays-bank-plc-ho-uk,Barclays Bank Plc Ho UK,United Kingdom,GBR,skandinaviska-enskilda-banken-stockholm-sweden-swe,Skandinaviska Enskilda Banken,Sweden,SWE,null,6990.0
223264,2924,the-bank-of-new-york-mellon-corp,The Bank of New York Mellon Corp.,"May 21, 2012","May 21, 2012",barclays-bank-plc-ho-uk,Barclays Bank Plc Ho UK,United Kingdom,GBR,skandinaviska-enskilda-banken-stockholm-sweden-swe,Skandinaviska Enskilda Banken,Sweden,SWE,null,8000.0
223265,2924,the-bank-of-new-york-mellon-corp,The Bank of New York Mellon Corp.,"May 21, 2012","May 21, 2012",barclays-bank-plc-ho-uk,Barclays Bank Plc Ho UK,United Kingdom,GBR,skandinaviska-enskilda-banken-stockholm-sweden-swe,Skandinaviska Enskilda Banken,Sweden,SWE,null,6990.0


In [0]:
#df = spark.sql("select icij_sar_id as id, begin_date as date, originator_bank_id as source, beneficiary_bank_id as target, amount_transactions as amount from download_transactions_map_csv")
#display(df.take(10))

id,date,source,target,amount
3297,"Mar 25, 2015",cimb-bank-berhad,barclays-bank-plc-london-england-gbr,5.689852347E7
3297,"Mar 30, 2015",cimb-bank-berhad,barclays-bank-plc-london-england-gbr,1.1623836125E8
2924,"Jul 5, 2012",barclays-bank-plc-ho-uk,skandinaviska-enskilda-banken-stockholm-sweden-swe,5000.0
2924,"Jun 20, 2012",barclays-bank-plc-ho-uk,skandinaviska-enskilda-banken-stockholm-sweden-swe,9990.0
2924,"May 31, 2012",barclays-bank-plc-ho-uk,skandinaviska-enskilda-banken-stockholm-sweden-swe,12000.0
2924,"May 29, 2012",barclays-bank-plc-ho-uk,skandinaviska-enskilda-banken-stockholm-sweden-swe,88.03
2924,"May 29, 2012",barclays-bank-plc-ho-uk,skandinaviska-enskilda-banken-stockholm-sweden-swe,13000.0
2924,"May 22, 2012",barclays-bank-plc-ho-uk,skandinaviska-enskilda-banken-stockholm-sweden-swe,6990.0
2924,"May 21, 2012",barclays-bank-plc-ho-uk,skandinaviska-enskilda-banken-stockholm-sweden-swe,8000.0
2924,"May 21, 2012",barclays-bank-plc-ho-uk,skandinaviska-enskilda-banken-stockholm-sweden-swe,6990.0


In [0]:
# banks = spark.sql("select distinct originator_bank_id as id from download_transactions_map_csv UNION select distinct beneficiary_bank_id as id from download_transactions_map_csv")
banks = csv.select("originator_bank_id").withColumnRenamed("originator_bank_id","id").union( \
        csv.select("beneficiary_bank_id").withColumnRenamed("beneficiary_bank_id","id")).distinct()

banks.write.format("org.neo4j.spark.DataSource") \
  .mode("Overwrite") \
  .option("url", dbutils.widgets.get("url")) \
  .option("authentication.basic.username", dbutils.widgets.get("user")) \
  .option("authentication.basic.password", dbutils.widgets.get("password")) \
  .option("node.keys", "id") \
  .option("labels", ":Entity").save()

In [0]:
from pyspark.sql import functions as func

transactions = csv.withColumnRenamed("originator_bank_id","source") \
                  .withColumnRenamed("beneficiary_bank_id","target") \
                  .groupBy("source","target").agg(func.sum("amount_transactions")) \
                  .withColumnRenamed("sum(amount_transactions)","amount")

display(transactions.take(10)) 

source,target,amount
chinatrust-commercial-bank,china-construction-bank-shenzhen-china-chn,1500185.0
marfin-popular-bank-public-co-ltd,hang-seng-bank-limited-hong-kong-hkg,2500000.0
national-bank-of-ras-al-khaimah,sumitomo-mitsui-banking-corporation-tokyo-japan-jpn,179930.0
primorye-bank,raiffeisen-bank-international-ag-vienna-austria-aut,1002659.75
national-bank-of-ras-al-khaimah,privatbank-ukraine-ukr,477018.6
sberbank,credit-suisse-ag-switzerland-che,7333581.45
danske-bank-as,sb-sberbank-jsc-almaty-kazakhstan-kaz,5810000.0
standard-chartered-bank-hong-kong-ltd,nordea-bank-danmark-as-copenhagen-denmark-dnk,970296.41
wells-fargo-bank-na,credit-suisse-ag-geneva-switzerland-che,2.0212764566E8
chinatrust-commercial-bank,industrial-and-comml-bank-of-china-fuzhou-china-chn,1254203.0


In [0]:
# transactions = spark.sql("select originator_bank_id as source, beneficiary_bank_id as target, sum(amount_transactions) as amount from download_transactions_map_csv group by source,target")

transactions.repartition(1).write.format("org.neo4j.spark.DataSource") \
.mode("Overwrite") \
.option("url", dbutils.widgets.get("url")) \
.option("authentication.basic.username", dbutils.widgets.get("user")) \
.option("authentication.basic.password", dbutils.widgets.get("password")) \
.option("relationship", "TRANSFERRED") \
.option("relationship.properties","amount") \
.option("relationship.save.strategy", "keys") \
.option("relationship.source.labels", ":Entity") \
.option("relationship.source.node.keys", "source:id") \
.option("relationship.source.save.mode", "Match") \
.option("relationship.target.labels", ":Entity") \
.option("relationship.target.node.keys", "target:id") \
.option("relationship.target.save.mode", "Match").save()

In [0]:
topReceivers = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", dbutils.widgets.get("url")) \
  .option("authentication.basic.username", dbutils.widgets.get("user")) \
  .option("authentication.basic.password", dbutils.widgets.get("password")) \
  .option("query", \
  "MATCH (s:Entity)-[tx:TRANSFERRED]->(t:Entity) RETURN s.id, t.id, sum(tx.amount) as total ORDER BY total DESC") \
  .load()

display(topReceivers)

s.id,total,t.id
amsterdam-trade-bank-nv,2.74702335212E9,rosbank-moscow-russia-rus
rigensis-bank-as,1.2011723472800002E9,ing-netherland-nv-netherlands-nld
ing-netherland-nv,1.19921965333E9,rigensis-bank-as-latvia-lva
jpmorgan-chase-bank,1.08274811202E9,deutsche-bank-ag-london-branch-gbr
as-expobank,8.880407306E8,credit-suisse-ag-zurich-switzerland-che
caledonian-bank-ltd,8.289382674300001E8,hongkong-and-shanghai-banking-corp-hong-kong-hkg
gazprombank,8.0265600676E8,jp-morgan-us-usa
as-expobank,7.698349E8,bank-soyuz-moscow-russia-rus
deutsche-bank-ag,6.81317607E8,usb-ag-singapore-sgp
societe-generale-private-banking,6.36828E8,societe-generale-bank-and-trust-singapore-sa-sgbt-singapore-singapore-sgp


In [0]:
pagerank = """
CALL gds.pageRank.stream( 
 { nodeProjection: 'Entity',relationshipProjection: 'TRANSFERRED',relationshipProperties: 'amount', relationshipWeightProperty: 'amount' }) 
YIELD nodeId, score 
RETURN gds.util.asNode(nodeId).id AS id, score AS score order by score desc
"""

ranks = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", dbutils.widgets.get("url")) \
  .option("authentication.basic.username", dbutils.widgets.get("user")) \
  .option("authentication.basic.password", dbutils.widgets.get("password")) \
  .option("query.count", 100) \
  .option("partitions", 1) \
  .option("schema.strategy", "string") \
  .option("query", pagerank) \
  .load().take(10)

display(ranks, ["id","score"])

score,id
4.0057680130004885,jsc-norvik-banka-riga-latvia-lva
2.6522245049476623,rosbank-moscow-russia-rus
2.602606767416,credit-suisse-ag-zurich-switzerland-che
2.3741217851638794,as-expobank-riga-latvia-lva
2.2314830509135772,dbs-bank-ltd-singapore-sgp
1.7248763620853422,ltb-bank-riga-latvia-lva
1.6822047531604767,seb-bank-tallinn-estonia-est
1.5058318549831793,hsbc-hong-kong-hkg
1.395442474351596,caledonian-bank-ltd-georgetown-cayman-islands-cym
1.2634113907814024,standard-chartered-bank-dubai-uae-are


In [0]:
clusters = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", dbutils.widgets.get("url")) \
  .option("authentication.basic.username", dbutils.widgets.get("user")) \
  .option("authentication.basic.password", dbutils.widgets.get("password")) \
  .option("query.count", 100) \
  .option("query", """
CALL gds.louvain.stream(
 { nodeProjection: 'Entity',relationshipProjection: 'TRANSFERRED',relationshipProperties: 'amount', relationshipWeightProperty: 'amount', includeIntermediateCommunities: true}) 
YIELD nodeId, intermediateCommunityIds as clusters
RETURN gds.util.asNode(nodeId).id, clusters
  """) \
  .load()

display(clusters.take(10))


gds.util.asNode(nodeId).id,clusters
hua-nan-commercial-bank-ltd,"List(1898, 2086)"
oversea-chinese-banking-corp,"List(88, 88)"
standard-bank-plc-london,"List(211, 211)"
harris-na,"List(1992, 2106)"
zions-first-national-bank-salt-lake-city-ut-usa,"List(4, 4)"
canara-bank-new-delhi-india-ind,"List(5, 2164)"
dms-bank-trust-ltd-cayman-islands-cym,"List(6, 6)"
petrocommerce-ojsc-bank-moscow-russia-rus,"List(7, 7)"
banque-privee-edmond-de-rothschild-suisse-sa-switzerland-che,"List(8, 1589)"
bbt-wilson-nc-usa,"List(9, 9)"
